In [1]:
import pybamm

parameters = pybamm.ParameterValues("Chen2020_composite")


# get the lithiation and delithiation functions

lithiation = parameters["Secondary: Negative electrode lithiation OCP [V]"]
delithiation = parameters["Secondary: Negative electrode delithiation OCP [V]"]


# define the new OCP function
def ocp(sto):
    return (lithiation(sto) + delithiation(sto)) / 2


# define hysteresis function
def hysteresis(sto):
    return (lithiation(sto) - delithiation(sto)) / -2


# add additional parameters
parameters.update(
    {
        "Secondary: Negative electrode OCP [V]": ocp,
        "Secondary: Negative particle hysteresis decay rate": 0.1,
        "Secondary: Negative electrode OCP hysteresis [V]": hysteresis,
        "Secondary: Negative particle hysteresis switching factor": 1,
    },
    check_already_exists=False,
)

model = pybamm.lithium_ion.SPM(
    {
        "open-circuit potential": (("single", "DCHS"), "single"),
        "particle phases": ("2", "1"),
    }
)


experiment = pybamm.Experiment(
    [
        ("Discharge at 1 C for 1 hour or until 2.5 V", "Rest for 15 minutes"),
        (
            "Charge at 1C until 4.2 V",
            "Hold at 4.2 V until 50 mA",
            "Rest for 15 minutes",
        ),
    ]
)


simulation = pybamm.Simulation(
    model, experiment=experiment, parameter_values=parameters
)
solution = simulation.solve(calc_esoh=False)

In [2]:
solution.plot(
    [
        "X-averaged negative electrode secondary hysteresis state",
        "Negative electrode secondary hysteresis state",
        "Negative electrode secondary open-circuit potential [V]",
        "Terminal voltage [V]",
        "X-averaged negative electrode secondary open-circuit potential [V]",
    ]
)

interactive(children=(FloatSlider(value=0.0, description='t', max=3.3439065949597495, step=0.03343906594959749…

In [3]:
import pybamm

parameters = pybamm.ParameterValues("Chen2020_composite")


# get the lithiation and delithiation functions

lithiation = parameters["Secondary: Negative electrode lithiation OCP [V]"]
delithiation = parameters["Secondary: Negative electrode delithiation OCP [V]"]


# define the new OCP function
def ocp(sto):
    return (lithiation(sto) + delithiation(sto)) / 2


# define hysteresis function
def hysteresis(sto):
    return (lithiation(sto) - delithiation(sto)) / -2


# add additional parameters
parameters.update(
    {
        "Secondary: Negative electrode OCP [V]": ocp,
        "Secondary: Negative particle hysteresis decay rate": 0.1,
        "Secondary: Negative electrode OCP hysteresis [V]": hysteresis,
        "Secondary: Negative particle hysteresis switching factor": 1,
    },
    check_already_exists=False,
)

model = pybamm.lithium_ion.SPM(
    {
        "open-circuit potential": (("single", "current sigmoid"), "single"),
        "particle phases": ("2", "1"),
    }
)


experiment = pybamm.Experiment(
    [
        ("Discharge at 1 C for 1 hour or until 2.5 V", "Rest for 15 minutes"),
        (
            "Charge at 1C until 4.2 V",
            "Hold at 4.2 V until 50 mA",
            "Rest for 15 minutes",
        ),
    ]
)


simulation = pybamm.Simulation(
    model, experiment=experiment, parameter_values=parameters
)
solution_sigmoid = simulation.solve(calc_esoh=False)

In [4]:
solution_sigmoid.plot(
    [
        # "X-averaged negative electrode secondary hysteresis state",
        # "Negative electrode secondary hysteresis state",
        "Negative electrode secondary open-circuit potential [V]",
        "Terminal voltage [V]",
        "X-averaged negative electrode secondary open-circuit potential [V]",
    ]
)

interactive(children=(FloatSlider(value=0.0, description='t', max=3.3455896030671757, step=0.03345589603067175…

In [5]:
pybamm.QuickPlot(
    [solution, solution_sigmoid],
    labels=["DCHS", "Sigmoid"],
    output_variables=[
        "Negative electrode secondary open-circuit potential [V]",
        "Terminal voltage [V]",
        "X-averaged negative electrode secondary open-circuit potential [V]",
    ],
).dynamic_plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=3.3455896030671757, step=0.03345589603067175…